## DELTA LAKE IMPLEMENTATION
- DLT tables are already Delta tables.
- DLT handles incremental logic internally via:
    - Streaming reads
    - Checkpoints
    - Deduplication (transaction_id)
    - Watermarks
- DLT handles Schema Evolution automatically if compatible (defined during table creation)

In [0]:
%sql
-- Verifying silver tables are Delta tables
DESCRIBE DETAIL workspace.default.silver_sales_transactions;


format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,5c73d290-062c-475a-9bba-e49e7ca4791d,workspace.default.silver_sales_transactions,Cleaned and standardized sales transactions,s3://dbstorage-prod-agfdu/uc/1f212cec-51df-430a-b621-cec5889adc83/a33e2734-9d3d-4e48-a1a9-7859526b7104/__unitystorage/catalogs/3819b248-df26-4723-9a04-edaad3495bd4/tables/b7ac83ce-68a6-4e0f-94e1-5d5923551878,2025-12-20T07:09:39.207Z,2025-12-20T07:10:37.000Z,List(),List(),1,181902,"Map(quality -> silver, spark.internal.pipelines.reconciliation_query_without_timetravel -> SELECT * FROM `workspace`.`default`.`__materialization_mat_850b5944_686a_41e6_b877_fada26febd32_silver_sales_transactions_1`, pipeline_internal.catalogType -> UNITY_CATALOG, delta.enableChangeDataFeed -> true, spark.sql.internal.pipelines.parentTableId -> 209f9b02-b579-4149-ba9c-e61c7e46c1f7, delta.enableDeletionVectors -> true, pipelines.pipelineId -> 850b5944-686a-41e6-b877-fada26febd32, pipeline_internal.enzymeMode -> Advanced, spark.internal.streaming_table.parentTable -> `workspace`.`default`.`silver_sales_transactions`, delta.enableRowTracking -> true, delta.rowTracking.materializedRowCommitVersionColumnName -> _row-commit-version-col-ec4c3c2f-7349-4487-bae0-f74dd7ce7854, pipelines.metastore.tableName -> `workspace`.`default`.`__materialization_mat_850b5944_686a_41e6_b877_fada26febd32_silver_sales_transactions_1`, delta.rowTracking.materializedRowIdColumnName -> _row-id-col-5fe3f8bc-3adb-4bd0-8f64-3cdcaa290790)",3,7,"List(appendOnly, changeDataFeed, deletionVectors, domainMetadata, invariants, rowTracking)","Map(numDeletionVectors -> 0, numRowsDeletedByDeletionVectors -> 0)",false


In [0]:
%sql

-- Demonstrate MERGE into Silver table

MERGE INTO workspace.default.silver_sales_transactions AS target
USING workspace.default.bronze_sales_transactions AS source
ON target.transaction_id = source.transaction_id

WHEN MATCHED THEN
  UPDATE SET
    target.quantity = source.quantity,
    target.unit_price = source.unit_price,
    target.total_amount = source.quantity * source.unit_price - source.discount,
    target.currency = source.currency

WHEN NOT MATCHED THEN
  INSERT *
;


In [0]:
%sql

-- Delta Time Travel

DESCRIBE HISTORY workspace.default.silver_sales_transactions;


In [0]:
%sql

-- Roll back to a previous version

RESTORE TABLE workspace.default.silver_sales_transactions
TO VERSION AS OF 3;


In [0]:
%sql
-- Optimize tables (OPTIMIZE)
-- Partition already handled logically via transaction_timestamp_utc partition during table definition
OPTIMIZE workspace.default.silver_sales_transactions
ZORDER BY (transaction_timestamp_utc, store_id);
